# 範例五 : 4大報表 
* (公開資訊戰)https://mops.twse.com.tw/mops/web/t164sb03 

In [5]:
import pandas as pd
import requests
import numpy as np
from io import StringIO
import datetime

In [6]:
class TwStockIdvs:
#     對照表
#     m_mapping = {'上市':'sii','上櫃':'otc'}
    base_url = 'https://mops.twse.com.tw/mops/web/'
    p_mapping = {'資產負債表':base_url+'t164sb03'
                ,'綜合損益表':base_url+'t164sb04'
                ,'現金流量表':base_url+'t164sb05'
                ,'權益變動表':base_url+'t164sb06'}
    old_p_mapping = {'資產負債表':base_url+'t05st31'
                ,'綜合損益表':base_url+'t05st34'
                ,'現金流量表':base_url+'t05st39'
                ,'權益變動表':base_url+'t05st38'}
    
    def __init__(self, co_id, year, season, purpose, \
                 isnew=None, p_mapping=p_mapping, old_p_mapping=old_p_mapping):
        cn_year = year if year < 1000 else year-1911
        eg_year = year if year > 1000 else year+1911
        ct_year = datetime.datetime.now().year
        
        if (eg_year>ct_year) or (purpose not in p_mapping):
            print('條件有誤，請重新輸入')
        else:
            isnew='false'
        '''
        elif eg_year==ct_year:
            isnew='true'
        elif eg_year<ct_year:
            isnew='false'
        '''
        p_mapping = p_mapping if cn_year>=102 else old_p_mapping
        
        self.co_id = str(co_id)
        self.year = str(cn_year)
        self.season = str(season)
        self.purpose = purpose
        self.isnew = isnew
        
        self.url = p_mapping[self.purpose] if self.purpose in p_mapping else None
    
    def add_raw(self):
        form = {"encodeURIComponent": "1",
                "step": "1",
                "firstin": "1",
                "off": "1",
                "keyword4": "",
                "code1": "",
                "TYPEK2": "",
                "checkbtn": "",
                "queryName": "co_id",
                "inpuType": "co_id",
                "TYPEK": "all",
                "isnew": self.isnew,
                "co_id": self.co_id,
                "year": self.year,
                "season": self.season}
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
                AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
            
        r = requests.post(self.url, form, headers=headers)
        r.encoding = 'utf8'
        print(f"{self.purpose}連結網址 :",r.url+f"?encodeURIComponent=1&step=1&off=1&queryName=co_id&TYPEK=all&inpuType=co_id&isnew={self.isnew}&firstin=1&co_id={self.co_id}&year={self.year}&season={self.season}")
        self.raw_data = pd.read_html(StringIO(r.text))
        
        
#     資產負債表
    def unify_baln(self):
#         ifrs 後
        if int(self.year)>=102:        
            data = self.raw_data[-2].drop(['Unnamed: 7_level_2', 'Unnamed: 8_level_2','Unnamed: 5_level_2','Unnamed: 6_level_2'], axis=1, level=2)
            data.columns = data.columns.droplevel([0,1])
            data = data.set_index('會計項目')
            data.index = [i[0] for i in data.index]
            data.columns = data.columns.remove_unused_levels()
#         ifrs 前
        else:
            data = x.raw_data[-2]
            head = list(set(data.iloc[3]))[1:]
            data.columns = list(data.iloc[2])
            data = data.drop([0,1,2,3])
            data = data.set_index('會計科目')
            time = list(set(data.columns))
            data.columns = pd.MultiIndex.from_product([time, head])
            data.index = list(data.index) 
        self.data = data
#     損益表
    def unify_incm(self):
#         ifrs 後
        if int(self.year)>=102:
            data = self.raw_data[-2].drop(['Unnamed: 9_level_2','Unnamed: 5_level_2','Unnamed: 5_level_3'], axis=1, level=2)
            data.columns = data.columns.droplevel([0,1])
            data = data.set_index('會計項目')
            data.index = [i[0] for i in data.index]
            data.columns = data.columns.remove_unused_levels()
#         ifrs 前
        else:
            data = x.raw_data[-2]
            head = list(set(data.iloc[3]))[1:]
            data.columns = list(data.iloc[2])
            data = data.drop([0,1,2,3])
            data = data.set_index('會計科目')
            time = list(set(data.columns))
            data.columns = pd.MultiIndex.from_product([time, head])
            data.index = list(data.index)
        self.data = data
        
    #     現金流量表
    def unify_socf(self):
#         ifrs 後
        if int(self.year)>=102:
            data = self.raw_data[-2].drop(['Unnamed: 3_level_2', 'Unnamed: 4_level_2', 'Unnamed: 5_level_2', 'Unnamed: 6_level_2','Unnamed: 3_level_3','Unnamed: 5_level_3','Unnamed: 6_level_3'], axis=1, level=2)
            data.columns = data.columns.droplevel([0,1])
            data = data.set_index('會計項目')
            data.index = [i[0] for i in data.index]
            data.columns = data.columns.remove_unused_levels()
#         ifrs 前
        else: #壞了
            data = x.raw_data[-2]
            head = list(set(data.iloc[3]))[1:]
            data.columns = list(data.iloc[2])
            data = data.drop([0,1,2,3])
            data = data.set_index('會計科目')
            time = list(set(data.columns))
            data.columns = pd.MultiIndex.from_product([time, head])
            data.index = list(data.index)
        self.data = data
    #     權益變動表
    def unify_sose(self):
#         ifrs 後
        if int(self.year)>=102:
            data = x.raw_data[-3]
            data.columns = list(data.iloc[0])
            data = data.set_index('會計項目')
            data.index = list(data.index)
            data = data[1:]
            data = data.dropna(axis='columns')
#         ifrs 前
        else: #壞了
            data = x.raw_data[-2]
            print(data)
            head = list(set(data.iloc[3]))[1:]
            data.columns = list(data.iloc[0])
            data = data.drop([0,1,2,3])
            #data = data.set_index('會計科目')
            time = list(set(data.columns))
            data.columns = pd.MultiIndex.from_product([time, head])
            data.index = list(data.index)
        self.data = data
    def save(self):
        name = f'./Data/{self.co_id}_{self.year}_{self.season}_{self.purpose}.csv'
        self.data.to_csv(name,encoding='utf-8-sig')

In [7]:
co_id = '2330'
year=108
season=4

In [8]:
x = TwStockIdvs(co_id, year, season, purpose='權益變動表')
x.add_raw()
x.unify_sose()
x.save()
x.data.head()

權益變動表連結網址 : https://mops.twse.com.tw/mops/web/t164sb06?encodeURIComponent=1&step=1&off=1&queryName=co_id&TYPEK=all&inpuType=co_id&isnew=false&firstin=1&co_id=2330&year=108&season=4


,普通股股本,股本合計,資本公積,法定盈餘公積,特別盈餘公積,未分配盈餘（或待彌補虧損）,保留盈餘合計,國外營運機構財務報表換算之兌換差額,透過其他綜合損益按公允價值衡量之金融資產未實現評價(損)益,避險工具之利益（損失）,其他,其他權益項目合計,歸屬於母公司業主之權益總計,非控制權益,權益總額
期初餘額,259303805,259303805,56315932,276033811,26907527,1073706503,1376647841,-12042347,-3429324,23601,-1843,-15449913,1676817665,678731,1677496396
提列法定盈餘公積,0,0,0,35113088,0,-35113088,0,0,0,0,0,0,0,0,0
提列特別盈餘公積,0,0,0,0,-16232421,16232421,0,0,0,0,0,0,0,0,0
普通股現金股利,0,0,0,0,0,-388955707,-388955707,0,0,0,0,0,-388955707,0,-388955707
因受領贈與產生者,0,0,3993,0,0,0,0,0,0,0,0,0,3993,13,4006


In [9]:
x = TwStockIdvs(co_id, year, season, purpose='資產負債表')
x.add_raw()
x.unify_baln()
x.save()
x.data.head()

資產負債表連結網址 : https://mops.twse.com.tw/mops/web/t164sb03?encodeURIComponent=1&step=1&off=1&queryName=co_id&TYPEK=all&inpuType=co_id&isnew=false&firstin=1&co_id=2330&year=108&season=4


108年12月31日          107年12月31日       
                                  金額      %           金額      %
流動資產                             NaN    NaN          NaN    NaN
現金及約當現金                  455399336.0  20.11  577814601.0  27.64
透過損益按公允價值衡量之金融資產－流動         326839.0   0.01    3504590.0   0.17
透過其他綜合損益按公允價值衡量之金融資產－流動  127396577.0   5.63   99561740.0   4.76
按攤銷後成本衡量之金融資產－流動            299884.0   0.01   14277615.0   0.68

In [10]:
x = TwStockIdvs(co_id, year, season, purpose='綜合損益表')
x.add_raw()
x.unify_incm()
x.save()
x.data.head()

綜合損益表連結網址 : https://mops.twse.com.tw/mops/web/t164sb04?encodeURIComponent=1&step=1&off=1&queryName=co_id&TYPEK=all&inpuType=co_id&isnew=false&firstin=1&co_id=2330&year=108&season=4


108年度                 107年度        
                      金額       %            金額       %
營業收入合計      1.069985e+09  100.00  1.031474e+09  100.00
營業成本合計      5.772869e+08   53.95  5.334875e+08   51.72
營業毛利（毛損）    4.926985e+08   46.05  4.979860e+08   48.28
未實現銷貨（損）益  -3.395000e+03    0.00  1.117880e+05    0.01
營業毛利（毛損）淨額  4.927019e+08   46.05  4.978743e+08   48.27

In [11]:
x = TwStockIdvs(co_id, year, season, purpose='現金流量表')
x.add_raw()
x.unify_socf()
x.save()
x.data.head()

現金流量表連結網址 : https://mops.twse.com.tw/mops/web/t164sb05?encodeURIComponent=1&step=1&off=1&queryName=co_id&TYPEK=all&inpuType=co_id&isnew=false&firstin=1&co_id=2330&year=108&season=4


,108年度,107年度
,金額,金額
營業活動之現金流量－間接法,NaN,NaN
繼續營業單位稅前淨利（淨損）,389845336.0,397510263.0
本期稅前淨利（淨損）,389845336.0,397510263.0
折舊費用,281411832.0,288124897.0
攤銷費用,5472409.0,4421405.0


# 結論 : 初步認定值不值得投資?
* (如何分析)https://www.stockfeel.com.tw/本益比、roe-及-roa-的意義/
* https://SHERRY-FREEDOMLIFESTYLE.COM/BEGINNERS-GUIDE-TO-FINANCIAL-STATEMENTS/#獲利性分析：公司的獲利能力